In [ ]:
import os
import sys

sys.path.append("..")
import language.decoder

import yaml
from keras import  models

from datasets.googlecc import PreProcessing
from datasets.common import get_dataset_metadata_cfg

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
with open("../configs/inception_lstm_preprocessed1.yaml") as fp:
    cfg = yaml.load(fp)

model_workspace_dir = os.path.join(cfg["workspace"]["directory"], cfg["dataset"]["name"], cfg["model"]["arch"])

dataset_cfg = get_dataset_metadata_cfg()

In [ ]:
dataset_preprocessor = PreProcessing(cfg, "inception", False, False)
training_generator, validation_generator, test_generator = dataset_preprocessor.get_keras_generators("inception")

In [ ]:
model = models.load_model(os.path.join(model_workspace_dir, 'weights.hdf5'))

In [ ]:
import pandas as pd
fp = open(dataset_cfg["data"]["googlecc"]["test_images_file"])

lines = fp.readlines()
number_test_examples = 100
generated_sentences = []
images = []
for i in range(number_test_examples):
    img = mpimg.imread(
        os.path.join(dataset_cfg["data"]["googlecc"]["dataset_path"],lines[i].strip())
    )
    imgplot = plt.imshow(img)
    plt.show()
    x, y = test_generator[i]
    sentence = language.decoder.greedy_decoder(
        model,
        x[0][0],
        dataset_preprocessor.get_word_dictionary(),
        dataset_preprocessor.get_id_dictionary(),
        40)
    generated_sentences.append(sentence)
    images.append(lines[i].strip())

data = {'Image ID': images, 'Model Output': generated_sentences }
df = pd.DataFrame(data)

In [ ]:
#BLEU analysis
from nltk.translate.bleu_score import sentence_bleu

dataset_workspace_dir = os.path.join(cfg["workspace"]["directory"], cfg["dataset"]["name"])
with open(os.path.join(dataset_workspace_dir, 'google_cc_processed_descriptions.txt')) as f:
    data = f.read()

img_captions = dict()
for line in data.split("\n"):
    if line.strip():
        tokens = line.split()
        image_id , image_desc = tokens[0],tokens[1:]
        img_captions[image_id] = image_desc


bleu_scores = []
score_sum = 0
for i in range(number_test_examples):
    image = df['Image ID'].iloc[i]
    candidate = df['Model Output'].iloc[i]
    candidate = candidate[1:-1]
    references = []
    references.append(img_captions[image.strip()])
    bleu_score = sentence_bleu(references, candidate, weights=(1, 0, 0, 0))
    bleu_scores.append(bleu_score)
    score_sum += bleu_score

df['Bleu Score'] = bleu_scores
df.to_csv(os.path.join(model_workspace_dir, "bleu_scores.csv"))

average_bleu = score_sum/number_test_examples
print("Average BLEU score", average_bleu)